# Playlist analyzer

Because of what happened with my account being flagged, this current code doesn't work. It was working perfectly until class, but I will have to find a workaround that doesn't use the api to find song data on the user's playlist. 

## Import the spotify api

In [67]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

# Define your credentials and redirection URI
SPOTIPY_CLIENT_ID = 'dda606479dd244a9b0948c01f155ac7a'
SPOTIPY_CLIENT_SECRET = 'd714739739a04b0fa662e521e1a85ff8'
SPOTIPY_REDIRECT_URI = 'http://localhost:3000'

# Create the Spotify client

spotify = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=SPOTIPY_CLIENT_ID,
    client_secret=SPOTIPY_CLIENT_SECRET,
    redirect_uri=SPOTIPY_REDIRECT_URI,
))

user_name = input("Enter your spotify ID: ")
playlists = spotify.user_playlists(user_name)
name = input("Enter playlist name: ")


# https://stackoverflow.com/questions/39086287/spotipy-how-to-read-more-than-100-tracks-from-a-playlist
def get_playlist_tracks(playlist_id):
    results = spotify.playlist_tracks(playlist_id)
    tracks = results['items']
    while results['next']:
        results = spotify.next(results)
        tracks.extend(results['items'])
    return tracks

playlist_id = 0
for i in playlists["items"] : 
    if i["name"] == name :
        playlist_id = i["id"]
playlist_data = get_playlist_tracks(playlist_id)


## Extract song data on each song

In [68]:
playlist = []
names = []

for i in playlist_data : 
    names.append(i["track"]["name"])
    playlist.append(spotify.audio_features(i["track"]["id"])[0])

print(playlist[1])


{'danceability': 0.782, 'energy': 0.787, 'key': 9, 'loudness': -2.533, 'mode': 1, 'speechiness': 0.0334, 'acousticness': 0.0125, 'instrumentalness': 5.97e-05, 'liveness': 0.112, 'valence': 0.533, 'tempo': 122.021, 'type': 'audio_features', 'id': '4eMKD8MRroxCqugpsxCCNb', 'uri': 'spotify:track:4eMKD8MRroxCqugpsxCCNb', 'track_href': 'https://api.spotify.com/v1/tracks/4eMKD8MRroxCqugpsxCCNb', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4eMKD8MRroxCqugpsxCCNb', 'duration_ms': 232131, 'time_signature': 4}


### Convert the new data into a dataframe including the name of each song

In [69]:
collumns = ["danceability", "energy", "key", "loudness", "mode", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "type", "id", "uri", "track_href", "analysis_url", "duration_ms", "time_signature"]
selected_columns = ["energy", "acousticness", "valence", "speechiness"]

df = pd.DataFrame(playlist, columns=collumns)
df = df[selected_columns]
df["name"] = names

# scaler = MinMaxScaler()
# scaler = scaler.fit(df[["tempo"]])

# df["tempo"] = pd.DataFrame(scaler.transform(df[["tempo"]]))
df.shape





(163, 5)

## Find the distance of each song to the central point of the playlist's cluster

In [70]:
model = KMeans(n_clusters=1)
model.fit(df[selected_columns])


centroid = model.cluster_centers_[0]
distances = []

for i in range(len(df)) :
    distance = cdist([df[selected_columns].iloc[i]], [centroid], metric="euclidean")
    distances.append(distance[0][0])

df["distance"] = distances

Q1 = df["distance"].quantile(0.25)
Q3 = df['distance'].quantile(0.75)

max = Q3 + (Q3-Q1)*2
print(max)

df.loc[df["distance"] > max]



0.7193555020707336


,energy,acousticness,valence,speechiness,name,distance
68,0.389,0.851,0.501,0.0280,Count on Me,0.729376
91,0.264,0.922,0.331,0.0322,All of Me,0.862850
96,0.296,0.934,0.120,0.0333,lovely (with Khalid),0.921084
123,0.350,0.827,0.279,0.0425,Arcade,0.753722
130,0.280,0.932,0.387,0.0434,When I Was Your Man,0.855310
142,0.303,0.694,0.139,0.0429,I Won't Give Up,0.724907


## Load in the data

In [71]:
df = pd.read_csv("data.csv")
df['energy'] = pd.to_numeric(df['energy'], errors='coerce')


/var/folders/k6/qgdgj9qj743ff3dn87fyv0yr0000gn/T/ipykernel_2575/3618754505.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data.csv")


## Find recommendations

In [73]:
recommendations = []
for i in range(len(df)) :
    distance = cdist([df[selected_columns].iloc[i].values], [centroid], metric="euclidean")
    if (distance < 0.2) :
        title = df.iloc[i]["title"]
        if ("(" not in title) : 
            recommendations.append(title)
    if len(recommendations) >= 50 :
        break

print(recommendations)



['Slowdown', 'Nothing Since Then', 'Learn to Fly', 'A Part of Myself', 'Light in Me', 'Light in me', 'Broken Inside', 'Kiss You', 'Heat', 'Good', 'Hope', 'Top', 'Combative Five', 'Cooked', 'School', 'Closterred', 'Frantic', 'one', 'Euphoria', 'Tasty', 'House Generation', 'Pendrive', 'Sting fells', 'Thin Bells', 'Wather', 'Downtown Joker', 'Hard Love', 'Mountain Goat', 'Red Magic', 'Ban Tay Lanh', 'Buoc Di', 'Canh En', 'Hoa Roi Cua Phat', 'Tim Em', 'Xa Em', 'Stay', 'Living Darfur', 'Electric Ride', 'Oxygenius', 'Down 2', 'Nocebo', 'Play Hard', 'Bad', 'Stay The Night', 'Secrets', 'Five Hours', 'Ten Feet Tall', 'Red Lights', 'On The Floor', 'Lost In Cheeseville']
